Import necessary libraries

In [ ]:
import requests
import json
import numpy as np
from collections import defaultdict
base_url = "https://livetiming.formula1.com/static/"

In [ ]:
season_urls = [
    "2024/Index.json",
    "2023/Index.json",
    "2022/Index.json",
    "2021/Index.json",
    "2020/Index.json",
    "2019/Index.json",
    "2018/Index.json"
]

sessions_by_circuit=defaultdict(list)

for season in season_urls:
    try:
        response = requests.get(f"{base_url}{season}")
        response.raise_for_status()
        season_data = json.loads(response.content.decode("utf-8-sig"))
        # print(json.dumps(season_data, indent=2))
        for meeting in season_data["Meetings"]:
            # print(json.dumps(meeting, indent=2))
            race_sessions=list(filter(lambda session: session["Type"] == "Race" and "Path" in session, meeting["Sessions"]))
            race_sessions_path=list(
                map(
                    lambda session: session["Path"]
                    , race_sessions
                )
            )

            circuit = meeting["Circuit"]["ShortName"]
            # print(json.dumps(race_sessions_path, indent=2))
            sessions_by_circuit[circuit].extend(race_sessions_path)
    except:
        continue

# pitstopseries_by_circuit=defaultdict(list)
pitlanetimecollection_by_circuit=defaultdict(list)
# select only those sessions where channel PitStopSeries is available
for circuit, sessions in sessions_by_circuit.items():
    for session in sessions:
        try:
            response = requests.get(f"{base_url}{session}Index.json")
            response.raise_for_status()
            data = json.loads(response.content.decode("utf-8-sig"))

            if "PitLaneTimeCollection" not in data['Feeds']:
                continue
            stream_path=f"{base_url}{session}{data['Feeds']['PitLaneTimeCollection']['StreamPath']}"
            response = requests.get(f"{base_url}{session}{data['Feeds']['PitLaneTimeCollection']['StreamPath']}")
            response.raise_for_status()
            stream_file = response.content.decode("utf-8-sig").split('\r\n')
            stream = list(map( lambda a : a[12:], stream_file ))
            # stream = list(filter( lambda a : '_deleted' not in a, stream ))
            stream = list(filter( lambda a : a != '', stream ))
            stream = list(map( lambda a : json.loads(a), stream ))
            for data in stream:
                for key, value in data["PitTimes"].items():
                    if key != "_deleted":
                        pitlanetimecollection_by_circuit[circuit].append(value)
            # print(json.dumps(stream))
        except:
            continue
    pitlanetimecollection_by_circuit[circuit] = list( filter( lambda a : a["Duration"].replace('.', '', 1).isdigit(), pitlanetimecollection_by_circuit[circuit] ) )
    pitlanetimecollection_by_circuit[circuit] = list( map( lambda a : float(a["Duration"]), pitlanetimecollection_by_circuit[circuit] ) )

pitlanetimecollection_by_circuit=dict(filter(lambda item: len(item[1]) > 0, pitlanetimecollection_by_circuit.items()))

calculate mean and sd

In [ ]:
circuit_pit_stat={}
for circuit, values in pitlanetimecollection_by_circuit.items():
    stats = np.array(values)
    normal_stats = stats[stats < 60]
    mean = np.around(np.mean(normal_stats), decimals=3)
    std = np.around(np.std(normal_stats), decimals=3)
    circuit_pit_stat[circuit]=dict(mean=mean, std=std)
    # print(circuit, mean, std,  stats[ stats > (mean + 5) ] )
with open('pit-time-stat.json', 'w') as json_file:
    json.dump(circuit_pit_stat, json_file, indent=2)
circuit_pit_stat

{'Catalunya': {'mean': 22.905, 'std': 3.032},
 'Spielberg': {'mean': 22.218, 'std': 3.497},
 'Silverstone': {'mean': 29.603, 'std': 2.512},
 'Hungaroring': {'mean': 23.754, 'std': 4.647},
 'Spa-Francorchamps': {'mean': 23.992, 'std': 3.637},
 'Zandvoort': {'mean': 22.58, 'std': 5.379},
 'Monza': {'mean': 26.001, 'std': 3.346},
 'Baku': {'mean': 19.774, 'std': 2.271},
 'Singapore': {'mean': 30.807, 'std': 3.553},
 'Austin': {'mean': 24.937, 'std': 2.816},
 'Mexico City': {'mean': 23.595, 'std': 3.413},
 'Interlagos': {'mean': 23.176, 'std': 4.383},
 'Las Vegas': {'mean': 22.52, 'std': 3.708},
 'Lusail': {'mean': 29.303, 'std': 4.164},
 'Yas Marina Circuit': {'mean': 22.602, 'std': 2.802},
 'Sakhir': {'mean': 25.806, 'std': 3.363},
 'Jeddah': {'mean': 23.3, 'std': 4.197},
 'Melbourne': {'mean': 23.545, 'std': 3.578},
 'Monte Carlo': {'mean': 24.819, 'std': 1.915},
 'Montreal': {'mean': 24.491, 'std': 2.082},
 'Suzuka': {'mean': 26.63, 'std': 7.611},
 'Imola': {'mean': 33.888, 'std': 7.46